In [ ]:
import sys
from pathlib import Path


def add_nearest_src_root(start: Path) -> Path:
    """Add the nearest ancestor containing a `src/` directory to sys.path.

    Args:
        start: Starting path for the upward search.

    Returns:
        The path inserted into sys.path.

    Raises:
        RuntimeError: If no ancestor with a `src` directory is found.
    """
    current: Path = start.resolve()

    for parent in (current, *current.parents):
        if (parent / "src").is_dir():
            sys.path.insert(0, str(parent))
            return parent

    raise RuntimeError("No ancestor directory containing 'src/' found")


PROJECT_ROOT: Path = add_nearest_src_root(start=Path.cwd())


**Import dependencies**

In [ ]:
import json
from datetime import datetime
from typing import Any

import polars as pl
import torch
from torch import Tensor

from src.data.load_utils import DataModule, load_datamodule_2010to2024
from src.models.models.TCN.interface import TCN
from src.models.utils import clean_model_saves, save_results

time: str = datetime.now().strftime(format="%Y%m%d-%H%M%S")
print(time)

20260103-023542


**Aquire the train and test data**

In [ ]:
data_module: DataModule = load_datamodule_2010to2024()
train_data: pl.DataFrame = data_module.extract_dataframe(
    inferior_percentile=0.0, superior_percentile=0.8
)
test_data: pl.DataFrame = data_module.extract_dataframe(
    inferior_percentile=0.0, superior_percentile=0.8
)

**Instantiate the model**

In [ ]:
feature_size: int = len(train_data.columns)
output_mask: Tensor = torch.tensor(
    data=[*(True for _ in range(12)), *(False for _ in range(19))]
)
lookback: int = 52

TCN_model: TCN = TCN(
    feature_size=feature_size,
    output_mask=output_mask,
    temporal_lookback=lookback,
    temporal_horizon=1,
)

**Evaluate the raw model on the test dataset**

In [ ]:
results: dict[str, Any] = TCN_model.evaluate(
    test_data=test_data,
    batch_size=16,
    drop_last=False,
)

print(json.dumps(obj=results, indent=2, sort_keys=True))

{
  "mae": 0.687454803423448,
  "mape": 1.88456212473916,
  "mse": 1.0003617007832426,
  "rmse": 0.8956050885307205,
  "smape": 1.7151300365274602
}


**Train the model on the train dataset**

In [6]:
train: bool = True
if train:
    TCN_model.fit(train_data=train_data)

Epoch 1/100: 100%|██████████| 30/30 [00:02<00:00, 10.26it/s, horizon=1, loss=0.843028, lr=1.91e-05]


[SmoothL1Loss (huber) loss] train: 0.342826, eval: 0.396259


Epoch 2/100: 100%|██████████| 30/30 [00:02<00:00, 10.97it/s, horizon=1, loss=0.519482, lr=3.97e-05]


[SmoothL1Loss (huber) loss] train: 0.343712, eval: 0.395529


Epoch 3/100: 100%|██████████| 30/30 [00:03<00:00,  9.25it/s, horizon=1, loss=0.245270, lr=7.17e-05]


...

[SmoothL1Loss (huber) loss] train: 0.183688, eval: 0.572710


Epoch 17/100: 100%|██████████| 30/30 [00:02<00:00, 10.32it/s, horizon=1, loss=0.150125, lr=2.96e-04]


[SmoothL1Loss (huber) loss] train: 0.173072, eval: 0.624531


Epoch 18/100:  37%|███▋      | 11/30 [00:01<00:01, 10.69it/s, horizon=1, loss=0.162566, lr=2.95e-04]


KeyboardInterrupt: 

**Evaluate the trained model on the test dataset**

In [ ]:
results: dict[str, Any] = TCN_model.evaluate(
    test_data=test_data,
    batch_size=16,
    drop_last=False,
)

print(json.dumps(obj=results, indent=2, sort_keys=True))

{
  "mae": 0.6937545958932463,
  "mape": 2.1142150997281908,
  "mse": 1.0053599720651454,
  "rmse": 0.9005714834153236,
  "smape": 1.7018879335243384
}


**Save the results**

In [ ]:
path: Path = PROJECT_ROOT / "models" / "TCN" / time
save_results(results=results, store_path=path)

**Save the model**

In [ ]:
path: Path = PROJECT_ROOT / "models" / "TCN" / time
TCN_model.save(store_path=path)

**Load the model**

In [ ]:
path: Path = PROJECT_ROOT / "models" / "TCN" / time
TCN_model.load(store_path=path)

**Clean TCN model saves**

In [ ]:
# remove every directory under model_path whiout a .keep file under it
# e.g. <model_path>/<dir>/.keep -> dont remove <dir>
clean: bool = False
if clean:
    path: Path = PROJECT_ROOT / "models" / "TCN"
    clean_model_saves(model_path=path)